In [1]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install langchain-experimental
!pip install langchain-openai 

## DDD strategic prompt

The main reason for creating this prompt is to guide architects in effectively identifying and delineating bounded contexts within a software project. It aims to streamline the strategic planning phase of DDD, enabling teams to produce a coherent model that aligns closely with the business objectives. This foundational work is crucial for developing software that is adaptable, scalable, and deeply integrated with the domain it serves, ultimately leading to more successful outcomes.

In the first part we will decompose the domain in subdomains. This is **problem space**.

## Domain-Driven Design definitions

In [2]:
ddd_strategic = {
  "domain" : {
    "definition" : "domain refers to the sphere of knowledge and activity around which a business, organization, or interest centers. It embodies the subject area to which the user applies a program is relevant. In simpler terms, it's the specific business or real-world problem that the software system aims to address or solve. The domain includes all the concepts, terms, activities, data, rules, and processes relevant to the business's problem area.",
    "example" : "The e-commerce domain focuses on the online retail of products and services, facilitating transactions between buyers and sellers through a digital platform. This domain encompasses various operations such as product listing, inventory management, order processing, payment processing, customer interaction, and logistics.",
    "format" : {
      "structure" : "<Name> ",
      "example" : "Ecommerce Ecosystem"
    }
  },
  "subdomain" : {
    "definition" : "a subdomain is a part of the domain that is a subset of the larger domain problem space. It represents a specific area of expertise or functionality within the overall domain. Subdomains are identified through the process of breaking down a complex domain into more manageable parts, each addressing a particular aspect of the problem domain. There are some classifications for subdomains: "
                   "Core Subdomain: The core subdomain is the heart of the business’s competitive advantage. It represents the primary domain that delivers the unique value that sets the business apart from its competitors. This is where the most significant business rules and processes that are critical to the company's success are implemented."
                   "Supporting Subdomain: Supporting subdomains handle important but not competitive aspects of a business. These domains support the core domain but do not constitute the primary business differentiator."
                   "Generic Subdomain: Generic subdomains are those parts of a system that can be implemented using off-the-shelf solutions or standard software practices without significant loss of effectiveness. These are typically common functionalities that are not specific to any particular business or industry.   " ,
    "example" : "The Order Management subdomain is responsible for all aspects of handling customer orders. This includes the process from the moment an order is placed by the customer through to its fulfillment and shipment, and potentially returns and refunds. It encompasses a range of functionalities such as order tracking, payment processing, order modification, and status updates.",
    "format" : {
      "structure" : "<Name> - <Classification>",
      "example" : "Order Management"
    }
  },
  "boundedContext" : {
    "definition" : "a Bounded Context is a central pattern that defines the limits of a specific domain model within which a domain-specific language is consistent and applicable. Essentially, it is a boundary within which a particular domain model is defined and applies, ensuring that all terms, concepts, and rules are unambiguous and coherent within it. This boundary can be physical, such as a specific service in a microservices architecture, or logical, such as a module or a team boundary within a larger software system.",
    "example" : "Order Processing",
    "format" : {
      "structure" : "< Name >",
      "example" : "The Order Processing bounded context focuses specifically on the lifecycle of an order from the moment it is placed by the customer until it is ready to be fulfilled. This includes order validation, payment processing, and order status updates. It is distinct from other aspects of Order Management, such as inventory checks or shipping, which may fall into different bounded contexts or integrate with this one through well-defined interfaces."
    }
  },
  "aggregate" : {
    "definition" : "Every Aggregate has a single root entity, known as the Aggregate Root. This root is the only member of the Aggregate that outside objects are allowed to hold references to.The Aggregate Root is responsible for enforcing the rules or invariants of the entire Aggregate. It acts as a gatekeeper, ensuring that the Aggregate's internal state changes are consistent with the domain rules.All external access to the objects within the Aggregate must flow through the Aggregate Root. This helps in maintaining consistency and integrity of the data within the Aggregate. ",
    "example" : "Order",
    "format" : {
      "structure" : "< Name >",
      "example" : "The Order entity acts as the Aggregate Root. It is the central point of access for any modifications related to an order. The Order entity contains all the essential information needed to place and process an order, such as customer details, shipping information, and a collection of OrderItems. "
    }
  }
}

## Domain Problem Statement

### Payment Gateway Definition
The Payment Gateway domain encompasses the infrastructure and services that facilitate online payment processing between merchants and customers. It acts as an intermediary, securely transmitting payment information from the customer to the merchant's bank account or payment processor. This domain includes handling authorization of transactions, encryption of payment details, fraud detection, and compliance with financial standards and regulations. Payment gateways are essential for e-commerce, enabling businesses to accept various forms of digital payments (such as credit cards, bank transfers, and digital wallets) while ensuring transaction security and integrity.  
  As a payment gateway company our main mission is to prevent fraud in our ecosystem. Our business differentiation is the small number of frauds we will keep investing money on that strategy. We want to create a payment gateway that is secure and reliable. We want to be the best in the market.

In [3]:
domain_statement = f"""
  The Payment Gateway domain encompasses the infrastructure and services that facilitate online payment processing between merchants and customers. It acts as an intermediary, securely transmitting payment information from the customer to the merchant's bank account or payment processor. This domain includes handling authorization of transactions, encryption of payment details, fraud detection, and compliance with financial standards and regulations. Payment gateways are essential for e-commerce, enabling businesses to accept various forms of digital payments (such as credit cards, bank transfers, and digital wallets) while ensuring transaction security and integrity.
  As a payment gateway company our main mission is to prevent fraud in our ecosystem. Our business differentiation is the small number of frauds we will keep investing money on that strategy. We want to create a payment gateway that is secure and reliable. We want to be the best in the market.
"""

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage,AIMessage
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

messages = [
    SystemMessage(content="You are a solution architect. Your main assignment is to create software documentation"),
    AIMessage(content="I want you to act as an IT Architect. I will provide some details about the functionality of an application or other digital product, and it will be your job to come up with ways to integrate it into the IT landscape. This could involve analyzing business requirements, performing a gap analysis and mapping the functionality of the new system to the existing IT landscape. Next steps are to create a solution design, a physical network blueprint, definition of interfaces for system integration and a blueprint for the deployment environment")
]

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

ddd_solution = PromptTemplate(
    input_variables=["ddd_definition","ddd_example","domain_statement"],
    template="""

        Domain = {ddd_definition}
        
        Example for a domain : {ddd_example}
        
        Your task is to perform the following actions: 
          1. Decompose the domain in subdomains
          2. Describe each subdomain in a few sentences.
          3. Classify each subdomain in using subdomain classification.
        
        Use the following format:
          Subdomain: <Name> 
          Type: <Classification>
          Summary: <Description>
          
        {domain_statement}  
        
        Give me list of subdomains classified. Put in order from the most important to less important subdomains.
        
    """
)

ddd_chain = LLMChain(llm=model,prompt=ddd_solution) 
ddd_definition = {ddd_strategic["domain"]["definition"]}
ddd_example = {ddd_strategic["domain"]["example"]}

final_chain = SequentialChain(chains=[ddd_chain],input_variables=["ddd_definition","ddd_example","domain_statement"],verbose=True)
response = final_chain.invoke({'ddd_definition': ddd_definition, 'ddd_example':ddd_example, 'domain_statement':domain_statement})
subdomain_response = response['text']
print(response['text'])



/var/folders/1n/_5t1ntj55s784hhr25d_9_sw0000gn/T/ipykernel_7586/3180485949.py:42: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  ddd_chain = LLMChain(llm=model,prompt=ddd_solution)




> Entering new SequentialChain chain...

> Finished chain.
Sure, here is the decomposition of the Payment Gateway domain into subdomains, classified and ordered from the most important to the least important:

1. **Subdomain: Fraud Detection and Prevention**
   - **Type:** Core
   - **Summary:** This subdomain focuses on identifying and mitigating fraudulent activities within the payment ecosystem. It involves the use of advanced algorithms, machine learning, and real-time monitoring to detect suspicious transactions and prevent fraud. Given the company's mission to prevent fraud, this is the most critical subdomain.

2. **Subdomain: Transaction Authorization**
   - **Type:** Core
   - **Summary:** This subdomain handles the approval or rejection of payment transactions. It involves verifying the customer's payment details, checking for sufficient funds, and ensuring compliance with the merchant's policies. This process is crucial for the smooth operation of the payment gateway.

3. 

In [5]:
bounded_context_prompt = PromptTemplate(
    input_variables=["bounded_context_definition","bounded_context_example","subdomain_response"],
    template="""

        Bounded Context = {bounded_context_definition}

        Example for a bounded context : {bounded_context_example}
        
        Your task is to perform the following actions:
          1. Analyse each subdomain and break it down into bounded contexts   
          2. List each bounded contexts in the following format:
                   <Name> 
                   <Description>
        
        Subdomains: {subdomain_response}
        
    """
)

ddd_chain = LLMChain(llm=model,prompt=bounded_context_prompt) 
bounded_context_definition = {ddd_strategic["boundedContext"]["definition"]}
bounded_context_example = {ddd_strategic["boundedContext"]["example"]}

final_chain = SequentialChain(chains=[ddd_chain],input_variables=["bounded_context_definition","bounded_context_example","subdomain_response"],verbose=True)
response = final_chain.invoke({'bounded_context_definition': bounded_context_definition, 'bounded_context_example':bounded_context_example, 'subdomain_response':subdomain_response})

print(response['text'])



> Entering new SequentialChain chain...

> Finished chain.
To break down each subdomain into bounded contexts, we need to identify the specific areas within each subdomain that have distinct responsibilities and can be encapsulated within their own boundaries. Here is the list of bounded contexts for each subdomain:

### 1. Fraud Detection and Prevention
- **Fraud Detection Algorithms**
  - **Description:** This context focuses on the development and maintenance of algorithms used to detect fraudulent activities. It includes machine learning models, rule-based systems, and anomaly detection techniques.
  
- **Real-Time Monitoring**
  - **Description:** This context is responsible for the real-time monitoring of transactions to identify suspicious activities. It involves setting up monitoring systems, dashboards, and alert mechanisms.
  
- **Fraud Case Management**
  - **Description:** This context handles the management of identified fraud cases, including investigation, reporting, a